In [30]:
#%matplotlib inline
import datetime
from dateutil import parser
import math
from dateutil.parser import parse
from geopandas import GeoDataFrame
import pandas as pd
import numpy as np
from shapely.geometry import Point
import json
import ast
import sys
import pickle
from __future__ import print_function
import os
import collections

folder_project = "/Users/sergiocamelo/Dropbox/Sergio-Joann/"
os.chdir(folder_project)
sys.path.insert(0, 'Code/VRPEngine/pyCode')

sys.path.insert(0, 'Code/VRPEngine/C++Engine')
sys.path.insert(0, 'Code/VRPEngine/pyCode/tsp')

import solver as solver
import distances as distances
import VRPClass
import road_distance
import json



In [31]:
days_analysis = 14
ts = 'Jan142018'
data_folder = 'StandardizedData/'
results_folder_ts = 'Results/'+ ts + "/" 
folder_instances = results_folder_ts+"instances"
solution_folder = results_folder_ts+'solution_routes/'

dim_farmers_file = 'dim_farmers_sample.csv'



In [32]:
routes_path = results_folder_ts+'parsed_routes.json'


In [33]:
# Extract routes
with open(routes_path) as infile:
    result = json.load(infile)


In [34]:
# Create dictionary of farmer to cluster
dim_farmers = pd.read_csv(results_folder_ts+"data_cleaning_results/dim_pickups.csv")
farmer_to_cluster = {}
for i,row in dim_farmers.iterrows():
    farmer_to_cluster[row['plot_id']] = row['cluster_id']
    
# # Create dictionary of farmer to cluster
# dim_farmers = pd.read_csv(data_folder+"dim_farmers.csv")
# farmer_to_cluster = {}
# for i,row in dim_farmers.iterrows():
#     farmer_to_cluster[row['farmer_id']+'-'+str(row['plot_number'])] = row['cluster_id']

In [35]:
# Create dictionary of farmer loads
farmer_load_dict = {}
for i,row in dim_farmers.iterrows():
    farmer_load_dict[row['plot_id']] = np.ceil(row["volume"] * 10)/10
    
# # Create dictionary of farmer loads
# volume = "high_volume"
# farmer_load_dict = {}
# for i,row in dim_farmers.iterrows():
#     farmer_load_dict[row['farmer_id']+'-'+str(row['plot_number'])] = np.ceil(row[volume] * 10)/10

In [37]:
# Create dictionary of truck loads
dim_trucks = pd.read_csv(results_folder_ts+"data_cleaning_results/dim_trucks.csv")
truck_load_dict = {}
for i,row in dim_trucks.iterrows():
    truck_load_dict[row['truck_id']] = float(row["capacity"])

In [39]:
# Create a truck to cluster dict
trucks = pd.read_csv(results_folder_ts+"data_cleaning_results/dim_trucks.csv")
truck_to_cluster = {}
for i,row in trucks.iterrows():
    truck_to_cluster[row['truck_id']] = row['cluster_id']

In [40]:
type_optimization = "daily"
routes = [r for r in result if r['type'] == type_optimization]
# Get rid of the polygons for the moment
for r in routes:
    if "polygon" in r:
        del r['polygon']

In [41]:
# Calculate total distance

total_dist = 0
total_time = 0
for route in routes:
    total_dist += route['distance']
    total_time += route['road_time']
total_fruit = 0
for route in routes:
    for farmer in route['route']:
        if farmer in farmer_load_dict:
            total_fruit += farmer_load_dict[farmer]

# Calculate percentage of relationships maintained
n_relations = 0
all_relations = 0
n_fruit = 0
for route in routes:
    for farmer in route['route']:
        if farmer in farmer_to_cluster:
            old_cluster = farmer_to_cluster[farmer]
            new_cluster = truck_to_cluster[route['truck']]
            if new_cluster != old_cluster:
                n_relations += 1
                n_fruit += farmer_load_dict[farmer]
            all_relations += 1
            

print("A total distance of %f"%(total_dist))

print("A total of %f relations changed"%(n_relations/(all_relations + 0.0)))
print("A total of %f fruit changed"%(n_fruit/(total_fruit + 0.0)))
#print("Total surplus increase is %f"%((np.sum(surplus.values()) - total_surplus_daily)/total_surplus_daily))


A total distance of 14332613.574852
A total of 0.000000 relations changed
A total of 0.000000 fruit changed


In [42]:
# Count number of days worked
days_worked = collections.defaultdict((lambda : collections.defaultdict(lambda : 0)))
for r in routes:
    days_worked[r['cluster']]['daily'] += 1

# Calculate efficiency
distance_dict = {
    'daily': total_dist
}
time_dict = {
    'daily': total_time
}

# Count quantities
quantities_served = collections.defaultdict((lambda : collections.defaultdict(lambda : 0)))
for r in routes:
    quantities_served[r['cluster']]['daily'] += r['load']

# Calculate load of trucks
load_trucks = collections.defaultdict((lambda : collections.defaultdict(lambda : [])))
for r in routes:
    load_trucks[r['truck']]['daily'] = load_trucks[r['truck']]['daily'] + [r['load']/truck_load_dict[r['truck']]]
for k in load_trucks.keys():
    load_trucks[k]['daily'] = (np.mean(load_trucks[k]['daily']),np.std(load_trucks[k]['daily']))
    
    
# Calculate load of clusters
load_clusters = collections.defaultdict((lambda : collections.defaultdict(lambda : 0)))
partial_sum = collections.defaultdict((lambda : collections.defaultdict(lambda : 0)))
for r in routes:
    load_clusters[truck_to_cluster[r['truck']]]['daily'] = load_clusters[truck_to_cluster[r['truck']]]['daily'] + r['load']
    partial_sum[truck_to_cluster[r['truck']]]['daily'] = partial_sum[truck_to_cluster[r['truck']]]['daily'] + truck_load_dict[r['truck']]
for k in load_clusters.keys():
    load_clusters[k]['daily'] = load_clusters[k]['daily']/(partial_sum[k]['daily']+0.0)

In [43]:
type_optimization = "spatial"
routes = [r for r in result if r['type'] == type_optimization]
# Get rid of the polygons for the moment
for r in routes:
    if "polygon" in r:
        del r['polygon']

In [44]:
# Calculate total distance

total_dist = 0
total_time = 0
for route in routes:
    total_dist += route['distance']
    total_time += route['road_time']
total_fruit = 0
for route in routes:
    for farmer in route['route']:
        if farmer in farmer_load_dict:
            total_fruit += farmer_load_dict[farmer]

# Calculate percentage of relationships maintained
n_relations = 0
all_relations = 0
n_fruit = 0
for route in routes:
    for farmer in route['route']:
        if farmer in farmer_to_cluster:
            old_cluster = farmer_to_cluster[farmer]
            new_cluster = truck_to_cluster[route['truck']]
            if new_cluster != old_cluster:
                n_relations += 1
                n_fruit += farmer_load_dict[farmer]
            all_relations += 1
            

print("A total distance of %f"%(total_dist))

print("A total of %f relations changed"%(n_relations/(all_relations + 0.0)))
print("A total of %f fruit changed"%(n_fruit/(total_fruit + 0.0)))
#print("Total surplus increase is %f"%((np.sum(surplus.values()) - total_surplus_daily)/total_surplus_daily))


A total distance of 8956604.561770
A total of 0.831016 relations changed
A total of 0.814746 fruit changed


In [45]:
# Count number of days worked
for r in routes:
    days_worked[truck_to_cluster[r['truck']]]['spatial'] = 0
for r in routes:
    days_worked[truck_to_cluster[r['truck']]]['spatial'] += 1

# Calculate efficiency
distance_dict.update({
    'spatial': total_dist
})
time_dict.update({
    'spatial': total_time
})

# Count quantities
for r in routes:
    quantities_served[truck_to_cluster[r['truck']]]['spatial'] = 0

for r in routes:
    quantities_served[truck_to_cluster[r['truck']]]['spatial'] += r['load']

# Calculate load of trucks
for r in routes:
    load_trucks[r['truck']]['spatial'] = []

for r in routes:
    load_trucks[r['truck']]['spatial'] = load_trucks[r['truck']]['spatial'] + [r['load']/truck_load_dict[r['truck']]]
for k in load_trucks.keys():
    load_trucks[k]['spatial'] = (np.mean(load_trucks[k]['spatial']),np.std(load_trucks[k]['spatial']))
    

/Users/sergiocamelo/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/sergiocamelo/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)


In [46]:
m_load_trucks = collections.defaultdict((lambda : collections.defaultdict(lambda : 0)))
for k in load_trucks.keys():
    m_load_trucks[k]['spatial'] = load_trucks[k]['spatial'][0]
    m_load_trucks[k]['daily'] = load_trucks[k]['daily'][0]

In [49]:
# Calculate the village of each middleman
from statistics import mode
import collections

dim_farmers_confirmed = pd.read_csv(data_folder + 'dim_farmers_confirmed.csv')
most_common = {}
for cluster_id, group in dim_farmers_confirmed.groupby('cluster_id'):
    counter=collections.Counter(group['house_village_name'])
    most_common[cluster_id] = counter.most_common(1)[0][0]

farmer_to_village = {}
dim_all_farmers = pd.read_csv(data_folder + 'dim_all_farmers.csv')
for i,row in dim_all_farmers.iterrows():
    farmer_to_village[row['farmer_id']+'-'+str(row['plot_number'])] = row['house_village_name']
dim_farmers_sample = pd.read_csv(data_folder + 'dim_farmers_sample.csv')
for i,row in dim_farmers_sample.iterrows():
    farmer_to_village[row['farmer_id']+'-'+str(row['plot_number'])] = row['house_village_name']


n_relations_village = 0
for route in routes:
    for farmer in route['route']:
        if farmer in farmer_to_cluster:
            old_cluster = farmer_to_cluster[farmer]
            new_cluster = truck_to_cluster[route['truck']]
            if new_cluster != old_cluster:
                if most_common[new_cluster] != farmer_to_village[farmer]:
                    n_relations_village += 1
            

In [51]:
# Efficiency
print("Total Fruit", total_fruit)
print("Time eff ", (time_dict['daily']-time_dict['spatial'])/time_dict['daily'])
print("Dist eff ", (distance_dict['daily']-distance_dict['spatial'])/distance_dict['daily'])
# Changed Relationships
import pandas as pd
print("Days worked less ",1-pd.DataFrame(days_worked).sum(axis=1)[1]/(pd.DataFrame(days_worked).sum(axis=1)[0]+0.0))
print("Average load trucks daily", np.nanmean([m_load_trucks[k]['daily'] for k in m_load_trucks]))
print("Average load trucks spatial", np.nanmean([m_load_trucks[k]['spatial'] for k in m_load_trucks]))

print("A total of %f fruit changed"%(n_fruit/(total_fruit + 0.0)))
print("A total of %f relations changed"%(n_relations/(all_relations + 0.0)))
print("A total of %f village relations changed"%(n_relations_village/(all_relations + 0.0)))


profit = 247*total_fruit*1000
costs_spatial = 7199.143394 * distance_dict['spatial']/1000
costs_daily = 7199.143394 * distance_dict['daily']/1000
print("Total profit reduction: %f"%((profit - costs_spatial)/(profit - costs_daily)-1))


Total Fruit 1624.7999999999984
Time eff  0.344343875598
Dist eff  0.375089231633
Days worked less  0.34036144578313254
Average load trucks daily 0.6603677664927664
Average load trucks spatial 0.826093966697415
A total of 0.814746 fruit changed
A total of 0.831016 relations changed
A total of 0.542246 village relations changed
Total profit reduction: 0.129812


In [52]:
pd.DataFrame([total_fruit,
              (time_dict['daily']-time_dict['spatial'])/time_dict['daily'],
            
            (distance_dict['daily']-distance_dict['spatial'])/distance_dict['daily'],
1-pd.DataFrame(days_worked).sum(axis=1)[1]/(pd.DataFrame(days_worked).sum(axis=1)[0]+0.0),
np.nanmean([m_load_trucks[k]['daily'] for k in m_load_trucks]),
np.nanmean([m_load_trucks[k]['spatial'] for k in m_load_trucks]),
(n_fruit/(total_fruit + 0.0)),
((profit - costs_spatial)/(profit - costs_daily)-1)
            
       ] ).transpose()

,0,1,2,3,4,5,6,7
0,1624.8,0.344344,0.375089,0.340361,0.660368,0.826094,0.814746,0.129812
